In [45]:
import time
import random
import os
import mxnet as mx
import numpy as np
np.set_printoptions(precision=2)
import argparse
import symbol

from skimage import io, transform, exposure, color

parser = argparse.ArgumentParser(description='mrf neural style')

parser.add_argument('--style-image', type=str)
parser.add_argument('--content-weight', nargs='+', type=float)
parser.add_argument('--style-weight', nargs='+', type=float)
parser.add_argument('--tv-weight', type=float)
parser.add_argument('--num-image', type=int)
parser.add_argument('--epochs', type=int)
parser.add_argument('--style-size', nargs='+', type=int)
parser.add_argument('--lr', type=float)
parser.add_argument('--model-name', type=str)
parser.add_argument('--num-res', type=int)
parser.add_argument('--num-rotation', type=int)
parser.add_argument('--num-scale', type=int)
parser.add_argument('--stride', type=int)
parser.add_argument('--patch-size', type=int)

args = parser.parse_args("--style-image ../images/bark.jpg --content-weight 1e-1 3e-1 3e-1 --style-weight 1 1 1 --tv-weight 1e-5 --num-image 2 --epochs 2 --style-size 512 512 --lr 3e3 --model-name ./models/bark_encoder_20 --num-res 3 --num-rotation 2 --num-scale 2 --stride 4 --patch-size 3".split())

VGGPATH = '../vgg19.params'
RESNETPATH= '../resnet18.params'
COCOPATH = '/home/ubuntu/data/train2014'

try:
    os.mkdir(args.model_name)
    os.mkdir('%s/data'%args.model_name)
    os.mkdir('%s/output'%args.model_name)
except:
    pass

def postprocess_img(im):
    im = im[0]
    im[0,:] += 123.68
    im[1,:] += 116.779
    im[2,:] += 103.939
    im = np.swapaxes(im, 0, 2)
    im = np.swapaxes(im, 0, 1)
    im[im<0] = 0
    im[im>255] = 255
    return im.astype(np.uint8)

def crop_img(im, size):
    im = io.imread(im)
    if len(im.shape) == 2:
        im = color.gray2rgb(im)
    if im.shape[0]*size[1] > im.shape[1]*size[0]:
        c = (im.shape[0]-1.*im.shape[1]/size[1]*size[0]) / 2
        c = int(c)
        im = im[c:-(1+c),:,:]
    else:
        c = (im.shape[1]-1.*im.shape[0]/size[0]*size[1]) / 2
        c = int(c)
        im = im[:,c:-(1+c),:]
    im = transform.resize(im, size)
    im = exposure.equalize_adapthist(im, kernel_size=(16,16), clip_limit=0.01)
    im *= 255
    return im

def preprocess_img(im):
    im = im.astype(np.float32)
    im = np.swapaxes(im, 0, 2)
    im = np.swapaxes(im, 1, 2)
    im[0,:] -= 123.68
    im[1,:] -= 116.779
    im[2,:] -= 103.939
    im = np.expand_dims(im, 0)
    return im

def get_mrf_executor(layer, patch_shape):
    patch_size = patch_shape[-1]
    data = mx.sym.Variable('conv')
    weight = mx.sym.Variable('weight')
    dist = mx.sym.Convolution(data=data, weight=weight, kernel=(patch_size, patch_size), num_filter=patch_shape[0], no_bias=True)
    dist_executor = dist.bind(args={'conv': layer, 'weight': mx.nd.zeros(patch_shape, mx.gpu())}, ctx=mx.gpu())
    return dist_executor

def get_tv_grad_executor(img, ctx, tv_weight):
    nchannel = img.shape[1]
    simg = mx.sym.Variable("img")
    skernel = mx.sym.Variable("kernel")
    channels = mx.sym.SliceChannel(simg, num_outputs=nchannel)
    out = mx.sym.Concat(*[
        mx.sym.Convolution(data=channels[i], weight=skernel,
                           num_filter=1,
                           kernel=(3, 3), pad=(1,1),
                           no_bias=True, stride=(1,1))
        for i in range(nchannel)])
    kernel = mx.nd.array(np.array([[0, -1, 0],
                                   [-1, 4, -1],
                                   [0, -1, 0]])
                         .reshape((1, 1, 3, 3)),
                         ctx) / 8.0
    out = out * tv_weight
    return out.bind(ctx, args={"img": img,
                               "kernel": kernel})


#sym, arg_params, aux_params = mx.model.load_checkpoint('../vgg19', 0000)

vgg_symbol = symbol.descriptor_symbol(args.num_res)
arg_names = vgg_symbol.list_arguments()
print(arg_names)
arg_dict = {}
pretrained = mx.nd.load(VGGPATH)
for name in arg_names:
    if name == "data":
        continue
    key = "arg:" + name
    if key in pretrained:
        arg_dict[name] = pretrained[key].copyto(mx.gpu())
del pretrained

img = None
args.style_size[0] = args.style_size[0] // 4 * 4
args.style_size[1] = args.style_size[1] // 4 * 4
size = [512, 512]
args.style_size = args.style_size[::-1]
rotations = [15*i for i in range(-args.num_rotation, args.num_rotation+1)]
scales = [1.05**i for i in range(-args.num_scale, args.num_scale+1)]

# extract patches
style_img = crop_img(args.style_image, args.style_size) 
patches = [[] for i in range(args.num_res)]
patches_normed = []
for s in scales:
    scaled = transform.rescale(style_img, s)
    arg_dict['data'] = mx.nd.zeros([len(rotations),3,scaled.shape[0],scaled.shape[1]], mx.gpu())
    for r in range(len(rotations)):
        arg_dict['data'][r:r+1] = preprocess_img(transform.rotate(scaled, rotations[r], mode='reflect'))
    #print(arg_dict.keys())
    vgg_executor = vgg_symbol.bind(ctx=mx.gpu(), args=arg_dict, grad_req='null')
    #vgg_executor = fe_sym.bind(ctx=mx.gpu(), args=arg_dict, aux_states=aux_params, grad_req='null')
    vgg_executor.forward()
    for l in range(args.num_res):
        tmp = vgg_executor.outputs[l].asnumpy()
        for ii in range(0, vgg_executor.outputs[l].shape[2]-args.patch_size+1, args.stride):
            for jj in range(0, vgg_executor.outputs[l].shape[3]-args.patch_size+1, args.stride):
                for r in range(len(rotations)):
                    patches[l].append(tmp[r,:,ii:ii+args.patch_size,jj:jj+args.patch_size])
for l in range(args.num_res):
    patches[l] = np.array(patches[l])
    tmp = np.linalg.norm(np.reshape(patches[l], [patches[l].shape[0], np.prod(patches[l].shape[1:])]), axis=1)
    norm = np.reshape(tmp, [tmp.shape[0],1,1,1])
    patches_normed.append(patches[l]/norm)
    patches[l] = mx.nd.array(patches[l], mx.gpu())



['data', 'conv1_1_weight', 'conv1_1_bias', 'conv1_2_weight', 'conv1_2_bias', 'conv2_1_weight', 'conv2_1_bias', 'conv2_2_weight', 'conv2_2_bias', 'conv3_1_weight', 'conv3_1_bias']


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [57]:
print(vgg_executor.arg_arrays[10])


[  3.20e-02  -3.09e-03   2.62e-02   3.34e-02  -1.62e-01   9.91e-04
   1.97e-02   8.87e-02  -1.67e-02   1.39e-02   6.34e-03   5.70e-02
   2.60e-03  -8.85e-02  -2.01e-02   5.24e-02   4.64e-02  -3.87e-02
   4.31e-02   4.70e-02  -1.73e-02  -5.11e-02  -5.79e-02   6.07e-02
   6.80e-04  -3.22e-02  -2.22e-02   5.58e-02  -1.69e-02   2.08e-02
  -1.13e-01   4.37e-02  -1.54e-01  -8.64e-02   3.97e-02  -3.47e-03
   6.98e-02  -2.54e-02  -5.02e-02   5.84e-02   5.48e-02  -5.19e-02
   4.75e-03  -5.42e-02   9.82e-03   1.02e-01   4.44e-02   2.47e-02
   1.45e-02   4.19e-02   1.90e-02   1.58e-02   1.50e-01  -7.73e-02
   6.14e-02   1.41e-01   3.34e-02   1.13e-02  -7.36e-02  -3.27e-02
  -6.35e-03   5.88e-02   3.38e-02   8.57e-03  -2.00e-02   1.78e-01
   3.54e-02   1.13e-02   2.50e-02  -5.55e-02  -9.21e-02   7.04e-02
  -1.41e-02  -3.63e-02   6.49e-02   3.35e-02   2.81e-02   3.35e-02
  -4.93e-02   1.04e-01   2.89e-02   3.36e-02  -9.32e-03  -5.36e-02
  -9.39e-03   7.79e-02  -5.85e-02  -5.65e-02   1.71e-04  -1.5

In [ ]:
arg_dict['data'] = mx.nd.zeros([1,3,size[0],size[1]], mx.gpu())
grad_dict = {"data": arg_dict["data"].copyto(mx.gpu())}
vgg_executor = vgg_symbol.bind(ctx=mx.gpu(), args=arg_dict, args_grad=grad_dict, grad_req='write')




In [2]:
import mxnet as mx

In [5]:
from mxnet.gluon.model_zoo.vision.resnet import resnet18_v1

In [41]:
resnet = resnet18_v1(pretrained=True)
print(resnet)
all_layers = resnet.get_internals()
net = all_layers
print(net)
new_args = dict({k:arg_params[k] for k in arg_params if 'dense' not in k})
resnet.features[4][1]['body']


ResNetV1(
  (features): HybridSequential(
    (0): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm(eps=1e-05, fix_gamma=False, axis=1, momentum=0.9)
    (2): Activation(relu)
    (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
    (4): HybridSequential(
      (0): BasicBlockV1(
        (body): HybridSequential(
          (0): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm(eps=1e-05, fix_gamma=False, axis=1, momentum=0.9)
          (2): Activation(relu)
          (3): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm(eps=1e-05, fix_gamma=False, axis=1, momentum=0.9)
        )
      )
      (1): BasicBlockV1(
        (body): HybridSequential(
          (0): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm(eps=1e-05, fix_gamma=False, axis=1, mome

AttributeError: 'ResNetV1' object has no attribute 'get_internals'

In [39]:
resnet.collect_params()

resnetv11_ (
  Parameter resnetv11_conv0_weight (shape=(64, 3, 7, 7), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_batchnorm0_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_batchnorm0_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_batchnorm0_running_mean (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_batchnorm0_running_var (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_stage1_conv0_weight (shape=(64, 64, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_stage1_batchnorm0_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_stage1_batchnorm0_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_stage1_batchnorm0_running_mean (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_stage1_batchnorm0_running_var (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter resnetv11_stage1_conv1_weight (shape=(64, 64, 3, 3), dtype=<class 'numpy.fl

In [38]:
'dense' in A;[]

True